# Intro to VectorBT

- [VectorBt](https://vectorbt.dev/) is a python library designed to conduct lightening fast backtests
    - It natively integrates Pandas and Numpy and uses Numba to speed up computations
- It also plugs straight into Plotly to produce some neat visualizations with minimal fuss
- [Comparison with other backtesting libraries here](https://www.qmr.ai/best-backtesting-library-for-python/)

**Pros:**
- Fastest library available
- Active development
- Developer is responsive to [feedback](https://github.com/polakowo/vectorbt/discussions)
- Integration with TA-Lib and Pandas-TA

**Cons:**
- Opinionated syntax, which makes it somewhat challenging to get used to the library
- Lack of documentation/tutorials
- Need to pay for [PRO](https://vectorbt.pro/) if you want the best experience (as well as adequate documentation)

In [210]:
# ! pip install vectorbt
import vectorbt as vbt

import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

### 1. Data loading

Vectorbt has a built-in adapter for [various APIs](https://vectorbt.dev/api/data/custom/) providing data

In [211]:
# docstring is probably the best way to get documentation
help(vbt.YFData)

Help on class YFData in module vectorbt.data.custom:

class YFData(vectorbt.data.base.Data)
 |  YFData(wrapper: vectorbt.base.array_wrapper.ArrayWrapper, data: Dict[Hashable, Union[pandas.core.series.Series, pandas.core.frame.DataFrame]], tz_localize: Union[NoneType, str, float, datetime.timedelta, datetime.tzinfo], tz_convert: Union[NoneType, str, float, datetime.timedelta, datetime.tzinfo], missing_index: str, missing_columns: str, download_kwargs: dict, **kwargs) -> None
 |  
 |  `Data` for data coming from `yfinance`.
 |  
 |  Stocks are usually in the timezone "+0500" and cryptocurrencies in UTC.
 |  
 |  !!! warning
 |      Data coming from Yahoo is not the most stable data out there. Yahoo may manipulate data
 |      how they want, add noise, return missing data points (see volume in the example below), etc.
 |      It's only used in vectorbt for demonstration purposes.
 |  
 |  Usage:
 |      * Fetch the business day except the last 5 minutes of trading data, and then update wi

In [212]:
crypto = vbt.YFData.download(
    ['BTC-USD', 'ETH-USD', 'XMR-USD'],
    interval='1d',
    missing_index='drop',
    start='2019-01-01',
    end = dt.datetime.today()
)
crypto

In [213]:
crypto_close = crypto.get('Close')
crypto_open = crypto.get('Open')

### 2. Indicators

For [technical analysis](https://vectorbt.dev/api/indicators/) believers VectorBT offers several [built-in indicators](https://vectorbt.dev/api/indicators/basic/) as well as indicators factory

In [214]:
rsi = vbt.RSI.run(crypto_close, window=20)
rsi

In [215]:
# to get values
rsi.rsi

rsi_window                        20                      
symbol                       BTC-USD    ETH-USD    XMR-USD
Date                                                      
2018-12-31 00:00:00+00:00        NaN        NaN        NaN
2019-01-01 00:00:00+00:00        NaN        NaN        NaN
2019-01-02 00:00:00+00:00        NaN        NaN        NaN
2019-01-03 00:00:00+00:00        NaN        NaN        NaN
2019-01-04 00:00:00+00:00        NaN        NaN        NaN
...                              ...        ...        ...
2023-05-11 00:00:00+00:00  48.375311  45.866858  51.854182
2023-05-12 00:00:00+00:00  43.790470  44.783582  44.333272
2023-05-13 00:00:00+00:00  44.921109  44.802059  43.385523
2023-05-14 00:00:00+00:00  46.296165  46.589138  44.601325
2023-05-15 00:00:00+00:00  44.164434  46.839684  40.666850

[1597 rows x 3 columns]

In [216]:
# VectorBT creates specific object instead of array to let you manipulate with it
entries = rsi.rsi_crossed_below(30)
exits = rsi.rsi_crossed_above(70)

In [217]:
# we will show in detail how the portfolio is built in the Portfolio construction section
crypto_pf = vbt.Portfolio.from_signals(
    crypto_close, entries, exits
)

In [218]:
crypto_pf.stats()

/tmp/ipykernel_130472/3290521750.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7f92d02c92d0>. Pass column to select a single column/group.



Start                         2018-12-31 00:00:00+00:00
End                           2023-05-15 00:00:00+00:00
Period                               1597 days 00:00:00
Start Value                                       100.0
End Value                                    136.443537
Total Return [%]                              36.443537
Benchmark Return [%]                         711.788684
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              66.653167
Max Drawdown Duration                 947 days 16:00:00
Total Trades                                        8.0
Total Closed Trades                            7.666667
Total Open Trades                              0.333333
Open Trade PnL                                 9.471565
Win Rate [%]                                  55.555556
Best Trade [%]                                41.774252
Worst Trade [%]                              -40

Metrics are poor... Probably the strategy is total shit.

Let's try a classical strategy based on two moving averages.  The indicator with a large number of periods is called a long average, the second line is called a short one. The strategy is to open and close a position depending on the behavior of these lines: if the short one crosses the long one from the bottom upwards, it gives a buy signal, if the crossing occurs from the top downwards - a sell signal.

In [219]:
def moving_averages(close, short_ma_window, long_ma_window):

    if short_ma_window >= long_ma_window:
        raise ValueError('The window of the short moving average should be less than the long one')
    
    short_ma = vbt.MA.run(close, window=short_ma_window)
    long_ma = vbt.MA.run(close, window=long_ma_window)

    trend = np.where(short_ma.ma_crossed_below(long_ma), 1, 0)
    trend = np.where(short_ma.ma_crossed_above(long_ma), -1, trend)

    return trend

In [220]:
ind = vbt.IndicatorFactory(
    class_name='moving_averages',
    short_name='mas',
    input_names=['close'],
    param_names=['short_ma_window', 'long_ma_window'],
    output_names=['value']
    ).from_apply_func(
        moving_averages,
        short_ma_window=10,
        long_ma_window=25
        )

In [221]:
results = ind.run(
    crypto_close,
    short_ma_window=20,
    long_ma_window=50
)

In [222]:
entries = results.value == 1
exits = results.value == -1

In [223]:
crypto_pf = vbt.Portfolio.from_signals(
    crypto_close, entries, exits
)

In [224]:
crypto_pf.stats(
    agg_func=None
).T

mas_short_ma_window                                 20  \
mas_long_ma_window                                  50   
symbol                                         BTC-USD   
Start                        2018-12-31 00:00:00+00:00   
End                          2023-05-15 00:00:00+00:00   
Period                              1597 days 00:00:00   
Start Value                                      100.0   
End Value                                   101.471062   
Total Return [%]                              1.471062   
Benchmark Return [%]                        632.262468   
Max Gross Exposure [%]                           100.0   
Total Fees Paid                                    0.0   
Max Drawdown [%]                             61.637687   
Max Drawdown Duration                737 days 00:00:00   
Total Trades                                        17   
Total Closed Trades                                 16   
Total Open Trades                                    1   
Open Trade PnL                               -1.065875   
Win Rate [%]                                     68.75   
Best Trade [%]                               38.893034   
Worst Trade [%]                             -40.000769   
Avg Winning Trade [%]                        14.546359   
Avg Losing Trade [%]                        -23.916129   
Avg Winning Trade Duration  25 days 21:49:05.454545454   
Avg Losing Trade Duration             78 days 00:00:00   
Profit Factor                                 1.018525   
Expectancy                                    0.158559   
Sharpe Ratio                                  0.257202   
Calmar Ratio                                  0.005424   
Omega Ratio                                   1.066543   
Sortino Ratio                                 0.354072   

mas_short_ma_window                                    \
mas_long_ma_window                                      
symbol                                        ETH-USD   
Start                       2018-12-31 00:00:00+00:00   
End                         2023-05-15 00:00:00+00:00   
Period                             1597 days 00:00:00   
Start Value                                     100.0   
End Value                                   94.078068   
Total Return [%]                            -5.921932   
Benchmark Return [%]                       1270.20359   
Max Gross Exposure [%]                          100.0   
Total Fees Paid                                   0.0   
Max Drawdown [%]                             75.92198   
Max Drawdown Duration               523 days 00:00:00   
Total Trades                                       16   
Total Closed Trades                                15   
Total Open Trades                                   1   
Open Trade PnL                              -0.771443   
Win Rate [%]                                     60.0   
Best Trade [%]                              30.927913   
Worst Trade [%]                            -44.634281   
Avg Winning Trade [%]                       16.383942   
Avg Losing Trade [%]                       -18.566166   
Avg Winning Trade Duration           33 days 02:40:00   
Avg Losing Trade Duration            56 days 00:00:00   
Profit Factor                                0.962013   
Expectancy                                  -0.343366   
Sharpe Ratio                                  0.22544   
Calmar Ratio                                -0.018249   
Omega Ratio                                  1.053579   
Sortino Ratio                                 0.32488   

mas_short_ma_window                                     
mas_long_ma_window                                      
symbol                                         XMR-USD  
Start                        2018-12-31 00:00:00+00:00  
End                          2023-05-15 00:00:00+00:00  
Period                              1597 days 00:00:00  
Start Value                                      100.0  
End Value                         

### 3. Plotting

In [225]:
vbt.settings.set_theme('dark')

In [226]:
# help(vbt.Portfolio.plot)

[Please see sources](https://github.com/polakowo/vectorbt/blob/master/vectorbt/generic/plots_builder.py)

In [227]:
crypto_pf.plot(
    subplots = [  # iterable where each element can be a subplot name (or 'all') or a tuple of name and a settings dict as in `PlotsBuilderMixin.subplots`.
      'cum_returns',
      'net_exposure',
      'drawdowns'
    ],   
    tags = None,
    column=None, # you may check pf.wrapper.columns
    group_by = True,
    silence_warnings = None,
    template_mapping = None,
    settings = None,
    filters = None,
    subplot_settings = None,
    show_titles = False,
    hide_id_labels = True,
    group_id_labels = None,
    make_subplots_kwargs = {
      'row_heights': [1, 0.33, 0.33],
      'vertical_spacing': 0.05
    },
    # **layout_kwargs
    paper_bgcolor = '#282c34',
    plot_bgcolor = '#282c34'

    
).show()

### 4. Order types, stop-loss, take-profit, shorting etc.

https://github.com/polakowo/vectorbt/blob/master/examples/StopSignals.ipynb

### 5. Portfolio construction

One of the most important concepts in vectorbt is broadcasting. Since vectorbt functions take time series as independent arrays, they need to know how to connect the elements of those arrays such that there is 1) complete information, 2) across all arrays, and 3) at each time step.

As a rule of thumb:

- If any array is a Pandas object, always produces a Pandas object
- If any array is two-dimensional, always produces a two-dimensional array
- If all arrays are constants or one-dimensional, always produces a one-dimensional array
- If any array is a DataFrame and this array is a one-dimensional NumPy array, broadcasts along columns
- If this array is a Series, always broadcasts along rows
- Lists and other sequences are converted to NumPy arrays prior to broadcasting

There are three main simulation modes in VectorBT

They are implemented through class methods

Remember that arguments passed to methods can be broadcasted - it greatly simplifies life and expands our toolbox!

#### 5.1 From orders
`Portfolio.from_orders` is the most straightforward and the fastest out of all simulation modes.
An order is a simple instruction that contains size, price, fees, and other information
(see `vectorbt.portfolio.enums.Order` for details about what information a typical order requires).

Thanks to broadcasting, we can pass any of the information as a 2-dim array, as a 1-dim array
per column or row, and as a constant. And we don't even need to provide every piece of information -
vectorbt fills the missing data with default constants, without wasting memory.

In [228]:
# set random weights for our crypto portfolio
# vectorbt does not support margin trading
# be careful with shorts
np.random.seed(42)
weights = np.random.uniform(low = 0, high = 1, size=crypto_close.shape)
weights = (weights.T / weights.sum(axis=1)).T

In [229]:
pf = vbt.Portfolio.from_orders(
    # 1. use prices
    close = crypto_close,   # current close as reference price, used for calculating return metrics, plotting etc.

    price = crypto_open,    # execution price, the most important parameter
                            # USE .shift() or .vbt.fshift(), so next day price would be used as execution price
                            # if the parameter is not passed, close is used (which is not correct)
                            # better take open as execution price or next day TWAP / POV / VWAP / (Open + Close)/2

    val_price = None,       # asset valuation price, used for target value and percentage
    ffill_val_price = None, # whether to track valuation price only if it's known

    # 2. set positions
    size = weights,                 # size to order

    size_type = 'targetpercent',    # "amount": amount of assets to trade / "targetamount": target amount of assets to hold (= target position)
                                    # "value": asset value to trade / "targetvalue": target asset value.
                                    # "percent": percentage of available resources to use in either direction (not to be confused with the percentage of position value!)
                                    # "targetpercent": target percentage of total value
                                    
    direction = 'both',             # longonly/shortonly/both
    lock_cash = None,               # whether to lock cash when shorting https://github.com/polakowo/vectorbt/issues/131

    # 3. construct portfolio
    cash_sharing = True,    # whether to share cash within the same group
    group_by = True,        # groups can be anything from positions or names of column levels, to a numpy array with actual groups.
                            # groups can be formed to share capital between columns (make sure to pass `cash_sharing=True`) or to compute metrics for a combined portfolio of multiple independent columns
    init_cash = None,       # initial capital

    # 4. set comissions
    fees = None,        # fees in percentage of the order value
    fixed_fees = None,  # fixed amount of fees to pay per order
    slippage = None,    # slippage in percentage of price

    # 5. model orders execution issues
    min_size = None,            # minimum size for an order to be accepted
    max_size = None,            # maximum size for an order
    size_granularity = None,    # granularity of the size
    reject_prob = None,         # order rejection probability
    raise_reject = None,        # whether to raise an exception if order gets rejected
    allow_partial = None,       # whether to allow partial fills

    # 6. provide details for post estimation analysis
    freq = '1d',            # used for annualization of metrics in case it cannot be parsed from `close`
    log = None,             # whether to log orders
    max_orders = None,      # size of the order records array
    max_logs = None,        # size of the log records array

    # 7. advanced stuff
    call_seq = None,            # default sequence of calls per row and group
    seed = None,                # seed to be set for both `call_seq` and at the beginning of the simulation
    attach_call_seq = None,     # whether to pass `call_seq` to the constructor. Makes sense if you want to analyze some metrics in the simulation order
    update_value = None,        # whether to update group value after each filled order
    )

In [230]:
# default parameters
vbt.settings['portfolio']

{'call_seq': 'default',
 'init_cash': 100.0,
 'size': inf,
 'size_type': 'amount',
 'fees': 0.0,
 'fixed_fees': 0.0,
 'slippage': 0.0,
 'reject_prob': 0.0,
 'min_size': 1e-08,
 'max_size': inf,
 'size_granularity': nan,
 'lock_cash': False,
 'allow_partial': True,
 'raise_reject': False,
 'val_price': inf,
 'accumulate': False,
 'sl_stop': nan,
 'sl_trail': False,
 'tp_stop': nan,
 'stop_entry_price': 'close',
 'stop_exit_price': 'stoplimit',
 'stop_conflict_mode': 'exit',
 'upon_stop_exit': 'close',
 'upon_stop_update': 'override',
 'use_stops': None,
 'log': False,
 'upon_long_conflict': 'ignore',
 'upon_short_conflict': 'ignore',
 'upon_dir_conflict': 'ignore',
 'upon_opposite_entry': 'reversereduce',
 'signal_direction': 'longonly',
 'order_direction': 'both',
 'cash_sharing': False,
 'call_pre_segment': False,
 'call_post_segment': False,
 'ffill_val_price': True,
 'update_value': False,
 'fill_pos_record': True,
 'row_wise': False,
 'flexible': False,
 'use_numba': True,
 'seed':

In [231]:
pf.stats(
    metrics = None,
    tags = None,
    column = None,
    group_by = None,
    agg_func = None,
    silence_warnings = None,
    template_mapping = None,
    settings = None,
    filters = None,
    metric_settings = None
).T

Start                           2018-12-31 00:00:00+00:00
End                             2023-05-15 00:00:00+00:00
Period                                 1597 days 00:00:00
Start Value                                         100.0
End Value                                      923.824709
Total Return [%]                               823.824709
Benchmark Return [%]                           711.788684
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                65.895995
Max Drawdown Duration                   734 days 00:00:00
Total Trades                                         2043
Total Closed Trades                                  2040
Total Open Trades                                       3
Open Trade PnL                                   4.704171
Win Rate [%]                                    52.696078
Best Trade [%]                                  31.213676
Worst Trade [%

Eleborate a bit more on orders execution

[See Portfolio() attributes](https://vectorbt.dev/api/portfolio/base/#vectorbt.portfolio.base.Portfolio.benchmark_rets)

In [232]:
# if you do not understand how your orders are executed

exit_trades = pf.exit_trades.records_readable   # each order that closes or removes from a position
entry_trades = pf.entry_trades.records_readable # each order that opens or adds to a position
positions = pf.positions.records_readable   # positions created from a sequence of entry or exit trades
trades = pf.trades.records_readable
orders = pf.orders.records_readable # capture information on filled orders
logs = pf.logs

#### 5.2 From signals
`Portfolio.from_signals` is centered around signals. It adds an abstraction layer on top of `Portfolio.from_orders`
to automate some signaling processes. For example, by default, it won't let us execute another entry signal
if we are already in the position. It also implements stop loss and take profit orders for exiting positions.
Nevertheless, this method behaves similarly to `Portfolio.from_orders` and accepts most of its arguments;
in fact, by setting `accumulate=True`, it behaves quite similarly to `Portfolio.from_orders`.

In a nutshell: this method automates some procedures that otherwise would be only possible by using
`Portfolio.from_order_func` while following the same broadcasting principles as `Portfolio.from_orders` -
the best of both worlds, given you can express your strategy as a sequence of signals. But as soon as
your strategy requires any signal to depend upon more complex conditions or to generate multiple orders at once,
it's best to run your custom signaling logic using `Portfolio.from_order_func`.

Useful when you deal with stop-losses / take-profits.

In [233]:
pf = vbt.Portfolio.from_signals(

    # 1. use prices
    close = crypto_close,
    price = None,
    val_price = None,
    ffill_val_price = None,
    open = None,        # used solely for stop signals
    high = None,        # used solely for stop signals
    low = None,         # used solely for stop signals


    # 2. set positions
    size = None,        # negative size is not allowed, you should express direction using signals
    size_type = None,   # target types not supported, 
                        # "percent" does not support position reversal, switch to a single direction or use `upon_opposite_entry` = 'close' to close the position first
                        
    direction = None,   # takes only effect if `short_entries` and `short_exits` are not set
    lock_cash = None,

    entries = entries,          # if `short_entries` and `short_exits` are not set: acts as a long signal if `direction` is `all` or `longonly`, otherwise short
                                # if `short_entries` or `short_exits` are set: acts as `long_entries`

    exits = exits,              # if `short_entries` and `short_exits` are not set: acts as a short signal if `direction` is `all` or `longonly`, otherwise long
                                # if `short_entries` or `short_exits` are set: acts as `long_exits`

    short_entries = None,       # boolean array of short entry signals
    short_exits = None,         # boolean array of short exit signals
    use_stops = None,           # whether to use stops
    sl_stop = None,             # percentage below/above the acquisition price for long/short position, note that 0.01 = 1%.
    sl_trail = None,            # whether `sl_stop` should be trailing
    tp_stop = None,             # percentage above/below the acquisition price for long/short position, note that 0.01 = 1%.
    stop_entry_price = None,    # which price to use as an initial stop price: "valprice"/"price"/"close"/"fillprice" (that is, slippage is already applied)

    stop_exit_price = None,     # which price to use when exiting a position upon a stop signal
                                # "stoplimit": stop price as from a limit order. If the stop was hit before, the opening price at the next bar is used. User-defined slippage is not applied.
                                # "stopmarket": stop price as from a market order. If the stop was hit before, the opening price at the next bar is used. User-defined slippage is applied.
                                # "price": default price
                                # "close": closing price

    accumulate = None,          # allows gradually increasing and decreasing positions by a size
                                # "disabled" or False: disable accumulation
                                # "both" or True: allow both adding to and removing from the position
                                # "addonly": allow accumulation to only add to the position
                                # "removeonly": allow accumulation to only remove from the position.

    # 3. resolve signals conflicts
    upon_long_conflict = None,      # what should happen if both entry and exit signals occur simultaneously
                                    # "ignore": ignore both signals
                                    # "entry": execute the entry signal
                                    # "exit": execute the exit signal
                                    # "adjacent": execute the adjacent signal, takes effect only when in position, otherwise ignores
                                    # "opposite": execute the opposite signal, takes effect only when in position, otherwise ignores

    upon_short_conflict = None,     # same as upon_long_conflict

    upon_dir_conflict = None,       # what should happen if both long and short entry signals occur simultaneously
                                    # "ignore": ignore both entry signals
                                    # "long": execute the long entry signal
                                    # "short": Execute the short entry signal
                                    # "adjacent": execute the adjacent entry signal, takes effect only when in position, otherwise ignores
                                    # "opposite": execute the opposite entry signal, takes effect only when in position, otherwise ignores

    upon_opposite_entry = None,     # what should happen if an entry signal of opposite direction occurs before an exit signal
                                    # "ignore": ignore the opposite entry signal
                                    # "close": close the current position
                                    # "closereduce": close the current position or reduce it if accumulation is enabled
                                    # "reverse": reverse the current position
                                    # "reversereduce": reverse the current position or reduce it if accumulation is enabled
    
    upon_stop_exit = None,          # how to exit the current position upon a stop signal
                                    # "close": close the current position
                                    # "closereduce": close the current position or reduce it if accumulation is enabled
                                    # "reverse": reverse the current position
                                    # "reversereduce": reverse the current position or reduce it if accumulation is enabled
    
    upon_stop_update = None,         # what to do with the old stop upon new acquisition
                                    # "keep": keep the old stop
                                    # "override": override the old stop, but only if the new stop is not NaN
                                    # "overridenan": override the old stop, even if the new stop is NaN

    
    # 4. construct portfolio
    cash_sharing = None,
    group_by = None,
    init_cash = None,

    # 5. set comissions
    fees = None,
    fixed_fees = None,
    slippage = None,

    # 6. model orders execution issues
    min_size = None,
    max_size = None,
    size_granularity = None,
    reject_prob = None,
    raise_reject = None,
    allow_partial = None,


    # 7. provide details for post estimation analysis
    freq = '1d',
    log = None,
    max_orders = None,
    max_logs = None,    # will be partially filled if exceeded. You might not be able to properly close the position if accumulation is enabled and `max_size` is too low.

    # 8. advanced stuff    
    call_seq = None,
    seed = None,
    attach_call_seq = None,
    update_value = None,

)

In [234]:
pf.stats(
    agg_func=None
).T

mas_short_ma_window                                 20  \
mas_long_ma_window                                  50   
symbol                                         BTC-USD   
Start                        2018-12-31 00:00:00+00:00   
End                          2023-05-15 00:00:00+00:00   
Period                              1597 days 00:00:00   
Start Value                                      100.0   
End Value                                   101.471062   
Total Return [%]                              1.471062   
Benchmark Return [%]                        632.262468   
Max Gross Exposure [%]                           100.0   
Total Fees Paid                                    0.0   
Max Drawdown [%]                             61.637687   
Max Drawdown Duration                737 days 00:00:00   
Total Trades                                        17   
Total Closed Trades                                 16   
Total Open Trades                                    1   
Open Trade PnL                               -1.065875   
Win Rate [%]                                     68.75   
Best Trade [%]                               38.893034   
Worst Trade [%]                             -40.000769   
Avg Winning Trade [%]                        14.546359   
Avg Losing Trade [%]                        -23.916129   
Avg Winning Trade Duration  25 days 21:49:05.454545454   
Avg Losing Trade Duration             78 days 00:00:00   
Profit Factor                                 1.018525   
Expectancy                                    0.158559   
Sharpe Ratio                                  0.257202   
Calmar Ratio                                  0.005424   
Omega Ratio                                   1.066543   
Sortino Ratio                                 0.354072   

mas_short_ma_window                                    \
mas_long_ma_window                                      
symbol                                        ETH-USD   
Start                       2018-12-31 00:00:00+00:00   
End                         2023-05-15 00:00:00+00:00   
Period                             1597 days 00:00:00   
Start Value                                     100.0   
End Value                                   94.078068   
Total Return [%]                            -5.921932   
Benchmark Return [%]                       1270.20359   
Max Gross Exposure [%]                          100.0   
Total Fees Paid                                   0.0   
Max Drawdown [%]                             75.92198   
Max Drawdown Duration               523 days 00:00:00   
Total Trades                                       16   
Total Closed Trades                                15   
Total Open Trades                                   1   
Open Trade PnL                              -0.771443   
Win Rate [%]                                     60.0   
Best Trade [%]                              30.927913   
Worst Trade [%]                            -44.634281   
Avg Winning Trade [%]                       16.383942   
Avg Losing Trade [%]                       -18.566166   
Avg Winning Trade Duration           33 days 02:40:00   
Avg Losing Trade Duration            56 days 00:00:00   
Profit Factor                                0.962013   
Expectancy                                  -0.343366   
Sharpe Ratio                                  0.22544   
Calmar Ratio                                -0.018249   
Omega Ratio                                  1.053579   
Sortino Ratio                                 0.32488   

mas_short_ma_window                                     
mas_long_ma_window                                      
symbol                                         XMR-USD  
Start                        2018-12-31 00:00:00+00:00  
End                          2023-05-15 00:00:00+00:00  
Period                              1597 days 00:00:00  
Start Value                                      100.0  
End Value                         

#### 5.3 From order function
`Portfolio.from_order_func` is the most powerful form of simulation. Instead of pulling information
from predefined arrays, it lets us define an arbitrary logic through callbacks. There are multiple
kinds of callbacks, each called at some point while the simulation function traverses the shape.
For example, apart from the main callback that returns an order (`order_func_nb`), there is a callback
that does preprocessing on the entire group of columns at once. For more details on the general procedure
and the callback zoo, see `vectorbt.portfolio.nb.simulate_nb`.

There is an even more flexible version available - `vectorbt.portfolio.nb.flex_simulate_nb` (activated by
passing `flexible=True` to `Portfolio.from_order_func`) - that allows creating multiple orders per symbol and bar.

This method has many advantages:
* Realistic simulation as it follows the event-driven approach - less risk of exposure to the look-ahead bias
* Provides a lot of useful information during the runtime, such as the current position's PnL
* Enables putting all logic including custom indicators into a single place, and running it as the data
 comes in, in a memory-friendly manner

But there are drawbacks too:
* Doesn't broadcast arrays - needs to be done by the user prior to the execution
* Requires at least a basic knowledge of NumPy and Numba
* Requires at least an intermediate knowledge of both to optimize for efficiency

In [235]:
# pf = vbt.Portfolio.from_order_func(
#     close = btc_price,
#     order_func_nb = lambda x: pass,
#     *order_args,
#     flexible = None,
#     init_cash = None,
#     cash_sharing = None,
#     call_seq = None,
#     segment_mask = None,
#     call_pre_segment = None,
#     call_post_segment = None,
#     pre_sim_func_nb: nb.PreSimFuncT = nb.no_pre_func_nb,
#     pre_sim_args = (),
#     post_sim_func_nb = nb.no_post_func_nb,
#     post_sim_args = (),
#     pre_group_func_nb = nb.no_pre_func_nb,
#     pre_group_args = (),
#     post_group_func_nb = nb.no_post_func_nb,
#     post_group_args = (),
#     pre_row_func_nb = nb.no_pre_func_nb,
#     pre_row_args = (),
#     post_row_func_nb = nb.no_post_func_nb,
#     post_row_args = (),
#     pre_segment_func_nb = nb.no_pre_func_nb,
#     pre_segment_args = (),
#     post_segment_func_nb = nb.no_post_func_nb,
#     post_segment_args = (),
#     post_order_func_nb = nb.no_post_func_nb,
#     post_order_args = (),
#     ffill_val_price = None,
#     update_value = None,
#     fill_pos_record = None,
#     row_wise = None,
#     use_numba = None,
#     max_orders = None,
#     max_logs = None,
#     seed = None,
#     group_by = None,
#     broadcast_named_args = None,
#     broadcast_kwargs = None,
#     template_mapping = None,
#     wrapper_kwargs = None,
#     freq = None,
#     attach_call_seq = None,
#     **kwargs -> PortfolioT:
# )

`Portfolio.from_holding` creates buy&hold equity curve.

## What else VectoBT can do:

- [Messaging using Telegram](https://vectorbt.dev/api/messaging/telegram/)
- Multitimeframes
- Parameter optimization

## Sources:

- [VectorBT docs](https://vectorbt.dev/)
- [VectorBT GitHub](https://github.com/polakowo/vectorbt) (pls grep discussions)
- [Jupyter Notebook examples](https://github.com/polakowo/vectorbt/tree/master/examples)
- [Great video tutorial for beginners](https://www.youtube.com/watch?v=JOdEZMcvyac&t=10552s)